
# Deep Learning Lab: Implementing a Multi-Layer Perceptron (MLP) from Scratch using NumPy

**Target audience:** 2nd Year B.Tech (Deep Learning Lab)  
**Goal:** Build a working MLP (2 → 4 → 1) **without** TensorFlow/PyTorch—only **NumPy**.

By the end of this notebook, you will understand:
- Forward propagation (prediction)
- Activation functions (ReLU, Sigmoid)
- Binary Cross-Entropy loss
- Backpropagation (gradients)
- Gradient Descent updates

---



## 0. Prerequisites (Quick Recap)

A neuron computes:

\[ z = XW + b \]

Then applies an activation:

\[ a = f(z) \]

An **MLP** stacks multiple such layers:

\[ X \rightarrow (W_1,b_1) \rightarrow a_1 \rightarrow (W_2,b_2) \rightarrow \hat{y} \]

---


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True, precision=4)



## 1. Dataset (Simple, small, and visual)

We'll start with a tiny binary classification dataset:  
Output is 1 **only** when both inputs are 1 (AND-like).

This is small enough to debug easily and perfect for learning.


In [ ]:
# Inputs (4 samples, 2 features)
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]], dtype=float)

# Labels (4 samples, 1 output)
y = np.array([[0],
              [0],
              [0],
              [1]], dtype=float)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("\nX:\n", X)
print("\ny:\n", y)



### Visual intuition (optional)

Points with label 1 are the "positive" class.


In [ ]:
plt.figure(figsize=(5,4))
for i in range(len(X)):
    plt.scatter(X[i,0], X[i,1], s=200)
    plt.text(X[i,0]+0.02, X[i,1]+0.02, f"y={int(y[i,0])}", fontsize=12)
plt.xlabel("x1")
plt.ylabel("x2")
plt.title("Toy Dataset (AND-like)")
plt.xlim(-0.2, 1.2)
plt.ylim(-0.2, 1.2)
plt.grid(True)
plt.show()



## 2. Define the MLP Architecture

We will build this network:

- **Input layer:** 2 features  
- **Hidden layer:** 4 neurons (ReLU)  
- **Output layer:** 1 neuron (Sigmoid)  

So the shapes are:
- \(W_1\): (2, 4) and \(b_1\): (1, 4)  
- \(W_2\): (4, 1) and \(b_2\): (1, 1)


In [ ]:
np.random.seed(42)

# Layer 1: 2 -> 4
W1 = np.random.randn(2, 4) * 0.5
b1 = np.zeros((1, 4))

# Layer 2: 4 -> 1
W2 = np.random.randn(4, 1) * 0.5
b2 = np.zeros((1, 1))

print("W1:", W1.shape, "b1:", b1.shape)
print("W2:", W2.shape, "b2:", b2.shape)



## 3. Activation Functions

- **ReLU** for hidden layer: \(\max(0, z)\)  
- **Sigmoid** for output (probability): \(\sigma(z)=\frac{1}{1+e^{-z}}\)

We also need derivatives for backprop:
- ReLU': 1 if z>0 else 0


In [ ]:
def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))



## 4. Loss Function (Binary Cross-Entropy)

For binary classification:
\[
L = -\frac{1}{N} \sum (y\log(\hat{y}) + (1-y)\log(1-\hat{y}))
\]

We add a tiny value (epsilon) to avoid log(0).


In [ ]:
def binary_cross_entropy(y_true, y_pred, eps=1e-8):
    y_pred = np.clip(y_pred, eps, 1 - eps)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))



## 5. Forward Propagation (Prediction)

Steps:
1. Hidden layer linear: \(z_1 = XW_1 + b_1\)
2. Hidden activation: \(a_1 = ReLU(z_1)\)
3. Output linear: \(z_2 = a_1W_2 + b_2\)
4. Output activation: \(\hat{y} = sigmoid(z_2)\)


In [ ]:
def forward_pass(X, W1, b1, W2, b2):
    # Hidden layer
    z1 = X @ W1 + b1
    a1 = relu(z1)
    
    # Output layer
    z2 = a1 @ W2 + b2
    y_hat = sigmoid(z2)
    
    cache = {"z1": z1, "a1": a1, "z2": z2, "y_hat": y_hat}
    return y_hat, cache

y_hat, cache = forward_pass(X, W1, b1, W2, b2)
print("Initial predictions (y_hat):\n", y_hat)
print("Initial loss:", binary_cross_entropy(y, y_hat))



## 6. Backpropagation (Gradients)

We compute gradients for:
- \(W_2, b_2\) using output error
- then propagate error to hidden layer to compute \(W_1, b_1\)

For sigmoid + BCE, a helpful simplification is:
\[
\frac{\partial L}{\partial z_2} = \hat{y} - y
\]

Then:
- \(dW_2 = a_1^T dz_2\)
- \(db_2 = \sum dz_2\)
- propagate: \(da_1 = dz_2 W_2^T\)
- apply ReLU derivative: \(dz_1 = da_1 \odot ReLU'(z_1)\)
- \(dW_1 = X^T dz_1\)
- \(db_1 = \sum dz_1\)


In [ ]:
def backward_pass(X, y, cache, W2):
    z1, a1, y_hat = cache["z1"], cache["a1"], cache["y_hat"]
    
    # Output layer gradient
    dz2 = y_hat - y                        # (N,1)
    dW2 = a1.T @ dz2                       # (4,1)
    db2 = np.sum(dz2, axis=0, keepdims=True)  # (1,1)
    
    # Hidden layer gradient
    da1 = dz2 @ W2.T                       # (N,4)
    dz1 = da1 * relu_derivative(z1)        # (N,4)
    dW1 = X.T @ dz1                        # (2,4)
    db1 = np.sum(dz1, axis=0, keepdims=True)  # (1,4)
    
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return grads

grads = backward_pass(X, y, cache, W2)
for k,v in grads.items():
    print(k, v.shape)



## 7. Training Loop (Gradient Descent)

We repeatedly:
1. Forward pass
2. Compute loss
3. Backprop gradients
4. Update parameters

Watch the loss go down!


In [ ]:
def train_mlp(X, y, hidden_size=4, lr=0.1, epochs=5000, seed=42, print_every=500):
    np.random.seed(seed)
    
    # Initialize
    W1 = np.random.randn(X.shape[1], hidden_size) * 0.5
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, 1) * 0.5
    b2 = np.zeros((1, 1))
    
    losses = []
    
    for epoch in range(1, epochs+1):
        # Forward
        y_hat, cache = forward_pass(X, W1, b1, W2, b2)
        loss = binary_cross_entropy(y, y_hat)
        losses.append(loss)
        
        # Backward
        grads = backward_pass(X, y, cache, W2)
        
        # Update
        W1 -= lr * grads["dW1"]
        b1 -= lr * grads["db1"]
        W2 -= lr * grads["dW2"]
        b2 -= lr * grads["db2"]
        
        if epoch % print_every == 0 or epoch == 1:
            print(f"Epoch {epoch:4d} | Loss: {loss:.4f}")
    
    params = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    return params, losses

params, losses = train_mlp(X, y, hidden_size=4, lr=0.1, epochs=5000, print_every=500)



### Plot Loss Curve


In [ ]:
plt.figure(figsize=(6,4))
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("Binary Cross-Entropy Loss")
plt.title("Training Loss Curve")
plt.grid(True)
plt.show()



## 8. Evaluate the Model

We will print final predictions and convert them to class labels using threshold 0.5.


In [ ]:
W1, b1, W2, b2 = params["W1"], params["b1"], params["W2"], params["b2"]

y_hat, _ = forward_pass(X, W1, b1, W2, b2)
pred = (y_hat >= 0.5).astype(int)

print("Final y_hat probabilities:\n", y_hat)
print("\nPredicted labels:\n", pred)
print("\nTrue labels:\n", y.astype(int))



## 9. Student Exercises (Do in Lab)

1. Change hidden neurons from 4 → 2 → 8. What happens to loss?
2. Set learning rate to 1.0. What happens? Why?
3. Replace ReLU with **tanh** in the hidden layer:
   - Implement `tanh` and its derivative
4. Try XOR dataset (more challenging):
   - X = [[0,0],[0,1],[1,0],[1,1]]
   - y = [[0],[1],[1],[0]]
   - Can your network learn it? (Hint: hidden layer helps)



## 10. (Optional) XOR Challenge Starter Code

Uncomment and try after you finish the main part.


In [ ]:
# # XOR dataset (uncomment to try)
# X_xor = np.array([[0,0],
#                   [0,1],
#                   [1,0],
#                   [1,1]], dtype=float)
# y_xor = np.array([[0],
#                   [1],
#                   [1],
#                   [0]], dtype=float)
#
# params_xor, losses_xor = train_mlp(X_xor, y_xor, hidden_size=4, lr=0.1, epochs=8000, print_every=800)
# W1, b1, W2, b2 = params_xor["W1"], params_xor["b1"], params_xor["W2"], params_xor["b2"]
# y_hat_xor, _ = forward_pass(X_xor, W1, b1, W2, b2)
# pred_xor = (y_hat_xor >= 0.5).astype(int)
# print("XOR probabilities:\n", y_hat_xor)
# print("XOR predicted labels:\n", pred_xor)
# print("XOR true labels:\n", y_xor.astype(int))
#
# plt.figure(figsize=(6,4))
# plt.plot(losses_xor)
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.title("XOR Training Loss")
# plt.grid(True)
# plt.show()



---
### Quick Teaching Tips (for the instructor)

- Make students **print shapes** after every major step (it prevents 80% of bugs).
- Explain: **forward = predict, backward = correct**
- Ask them: “What happens if activation is removed?” (They’ll *see* why non-linearity matters.)

✅ End of Notebook
